Upload .mat file into Python and see its structure | Загрузить .mat файл в Python и посмотреть его структуру

In [ ]:
import numpy as np
from scipy.io import loadmat

data = loadmat('ex4data1.mat')
data

Let's check the dimensionality of the Data Frame | Давайте определим размерность загруженных данных 'X' и 'y'

In [ ]:
import pandas as pd
data['X'].shape, data['y'].shape

Let's have a look into any array of 'X' | Давайте посмотрим на структуру любой строки в массиве 'X', например строки 4999  

In [ ]:
data['X'][4999]

Let's assign variable "X" and "y" to our arrays | Давайте присвоим переменные "X" and "y" для наших двух массивов

In [ ]:
X = data['X']
y = data['y']

X.shape, y.shape

Let's transform array "y" into a new variable "y_onehot" using OneHotEncoder | Теперь преобразуем массив "y" в переменную "y_onehot" с помощью OneHotEncoder

In [55]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

Let's compare "y" and "y_onehot" using OneHotEncoder | Теперь сравним "y" и "y_onehot"

In [58]:
y[4999], y_onehot[0,:]

(array([9], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]))